# Vehicle Data Cleansing Pipeline

This notebook processes vehicle data from `vehicles_data.json` through a cleaning pipeline. We used to have multiple scripts for different cleaning steps that were all combined in here in a sensible order. The final result will be `vehicles_info.yaml` where the scraped vehicle information is cleaned, properly translated (as even the english version of the car selling website did still show german words), and token-limited (for BERT).

## 1. Setup and Imports

Import all necessary libraries and set up logging for the data pre-processing pipeline.

In [1]:
import json
import os
import re
import logging
from typing import Iterable, Dict, Any, Tuple
from collections import defaultdict

import yaml
from langdetect import detect, LangDetectException
from deep_translator import GoogleTranslator
from transformers import BertTokenizer

# Setup logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Clear existing handlers
if logger.handlers:
    logger.handlers.clear()

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(logging.Formatter("%(levelname)s: %(message)s"))
logger.addHandler(stream_handler)

print("Libraries imported successfully!")

Libraries imported successfully!


/storage/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Load Original Data

Load the original vehicle data from `vehicles_data.json` and perform initial validation.

In [2]:
# Define file paths
base_dir = os.path.abspath(os.path.join('../../data'))
input_file = os.path.join(base_dir, "vehicles_data.json")
output_file = os.path.join(base_dir, "vehicles_info.yaml")

# Load original data
print(f"Loading data from: {input_file}")
with open(input_file, "r", encoding="utf-8") as read_file:
    original_data = json.load(read_file)

if not original_data:
    raise ValueError("The provided file is empty—nothing to clean.")

print(f"Loaded {len(original_data)} vehicle entries")
print(f"Sample entry URL: {list(original_data.keys())[0]}")

Loading data from: /storage/courses/ds_workflow/car-web-scraping/data/vehicles_data.json
Loaded 2600 vehicle entries
Sample entry URL: https://autobid.de/en/item/mini-cooper-s-cabrio-3111666/details


## 3. Step 1: Data Cleaning and Deduplication

Clean the data by removing duplicate phrases from details_text that already appear in details_list or information_dict.  
Also scraped items that doesn't contain all information fields that we expect are skipped, as sometimes not only vehicles are on sale but also vehicle parts that don't have all the information we'd expect. (And we don't want them.)

In [3]:
def _dedupe_details_text(details_text: str,
                         details_list: Iterable[str],
                         info_values: Iterable[str]) -> str:
    """Remove duplicate phrases from details_text that appear in details_list or info_values."""
    phrases = {p.lower().strip() for p in details_list if p.strip()}
    phrases.update(str(v).lower().strip() for v in info_values if v)

    for phrase in sorted(phrases, key=len, reverse=True):
        if not phrase:
            continue
        pattern = re.compile(rf"\b{re.escape(phrase)}\b[.,;:]?", re.IGNORECASE)
        details_text = pattern.sub("", details_text)

    # Remove extra whitespace and punctuation
    details_text = re.sub(r"\s{2,}", " ", details_text)
    details_text = re.sub(r"[;,]{2,}", ";", details_text)
    details_text = re.sub(r"^[\s;,.]+|[\s;,.]+$", "", details_text)

    return details_text.strip()

# Apply deduplication
cleaned_data = {}
skipped_count = 0
modified_count = 0

for url, car_dict in original_data.items():
    details_list = car_dict.get("details_list", [])
    info_dict = car_dict.get("information_dict", {})
    details_text = car_dict.get("details_text", "")

    # Skip incomplete rows
    if not (all(details_list) and all(info_dict.values()) and details_text):
        # logger.warning("Skipped incomplete entry for %s", url)
        skipped_count += 1
        continue

    cleaned_text = _dedupe_details_text(
        details_text,
        details_list,
        info_dict.values()
    )

    if cleaned_text != details_text:
        # logger.info("Modified details_text for %s", url)
        modified_count += 1

    cleaned_entry = dict(car_dict)
    cleaned_entry["details_text"] = cleaned_text
    cleaned_data[url] = cleaned_entry

print(f"\nStep 1 Complete:")
print(f"- Processed: {len(cleaned_data)} entries")
print(f"- Skipped: {skipped_count} incomplete entries")
print(f"- Modified: {modified_count} entries")


Step 1 Complete:
- Processed: 2510 entries
- Skipped: 90 incomplete entries
- Modified: 514 entries


## 4. Step 2: Text Translation

Translate the German texts that are sometimes in the details to English using Google Translate.  
For some reason this sometimes skipps texts so we still get the german version.

In [4]:
# Translation constants
_MAX_CHARS_PER_REQUEST = 4500  # Google free web translate limit

def _translate(text: str) -> str:
    """Translate text to English using Google Translate."""
    if not text or not text.strip():
        return text

    try:
        try:
            lang = detect(text)
        except LangDetectException:
            lang = "unknown"
        if lang == "en":
            return text

        translator = GoogleTranslator(source="auto", target="en")

        if len(text) <= _MAX_CHARS_PER_REQUEST:
            return translator.translate(text)

        # Chunk long passages
        chunks = [
            text[i : i + _MAX_CHARS_PER_REQUEST]
            for i in range(0, len(text), _MAX_CHARS_PER_REQUEST)
        ]
        translated_chunks = []
        for chunk in chunks:
            translated_chunks.append(translator.translate(chunk))
        return "".join(translated_chunks)

    except Exception as exc:
        logger.error("Google translation failed: %s – returning original text", exc)
        return text

# Apply translation
translated_data = {}
translation_count = 0

for url, car in cleaned_data.items():
    translated_entry = dict(car)
    
    free_text = car.get("details_text", "")
    if not free_text:
        translated_data[url] = translated_entry
        continue

    translated_text = _translate(free_text)
    translated_entry["details_text"] = translated_text

    if translated_text != free_text:
        # logger.info("Translated: %s", url)
        translation_count += 1

    translated_data[url] = translated_entry

print(f"\nStep 2 Complete:")
print(f"- Processed: {len(translated_data)} entries")
print(f"- Translated: {translation_count} entries")


Step 2 Complete:
- Processed: 2510 entries
- Translated: 1069 entries


## 5. Step 3: Token Limit Enforcement

BERT has a Token Limit of 512 Tokens. Those Tokens aren't allowed to be exceeded by the combination of vehicle information and query together. So we decided that 400 tokens at maximum for the vehicle information might work for us and stripped the vehicle details (`details_text`) for some cars. This will leave us with 109 tokens of legnth for the search query (plus 3 tokens for start, separation and stop BERT uses).

In [5]:
# Initialize BERT tokenizer
print("Loading BERT tokenizer...")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
TOKEN_LIMIT = 400  # Set your desired token limit

def extract_non_details_text(vehicle_data: Dict[str, Any]) -> str:
    """Extract details_list and information_dict text (parts that won't be cut)."""
    text_parts = []
    
    if 'details_list' in vehicle_data:
        text_parts.extend(vehicle_data['details_list'])
    
    if 'information_dict' in vehicle_data:
        for key, value in vehicle_data['information_dict'].items():
            text_parts.append(f"{key}: {value}")
    
    return ' '.join(text_parts)

def get_total_token_count(vehicle_data: Dict[str, Any], tokenizer) -> int:
    """Get total token count for a vehicle listing."""
    text_parts = []
    
    if 'details_text' in vehicle_data:
        text_parts.append(vehicle_data['details_text'])
    
    if 'details_list' in vehicle_data:
        text_parts.extend(vehicle_data['details_list'])
    
    if 'information_dict' in vehicle_data:
        for key, value in vehicle_data['information_dict'].items():
            text_parts.append(f"{key}: {value}")
    
    full_text = ' '.join(text_parts)
    return len(tokenizer.encode(full_text, add_special_tokens=False))

def truncate_by_sentences(details_text: str, fixed_text: str, token_limit: int, tokenizer) -> str:
    """Truncate details_text by complete sentences."""
    if not details_text:
        return details_text
    
    # Find sentence separators (. and ;)
    sentence_pattern = r'[.;]'
    sentences = re.split(sentence_pattern, details_text)
    
    # If no sentence separators found, return original text for word-based truncation
    if len(sentences) <= 1:
        return details_text
    
    # Rebuild sentences with their separators
    separators = re.findall(sentence_pattern, details_text)
    reconstructed_sentences = []
    
    for i, sentence in enumerate(sentences[:-1]):
        if i < len(separators):
            reconstructed_sentences.append(sentence + separators[i])
    
    # Add last sentence if it exists and doesn't end with separator
    if sentences[-1].strip():
        reconstructed_sentences.append(sentences[-1])
    
    # Find the longest valid truncation
    truncated_details = ""
    for sentence in reconstructed_sentences:
        candidate_details = truncated_details + sentence
        candidate_full_text = ' '.join([candidate_details, fixed_text]).strip()
        
        if len(tokenizer.encode(candidate_full_text, add_special_tokens=False)) <= token_limit:
            truncated_details = candidate_details
        else:
            break
    
    return truncated_details.strip()

def truncate_by_words(details_text: str, fixed_text: str, token_limit: int, tokenizer) -> str:
    """Truncate details_text by 10-word chunks."""
    if not details_text:
        return details_text
    
    words = details_text.split()
    truncated_details = ""
    
    # Try chunks of 10 words
    for i in range(0, len(words), 10):
        candidate_words = words[:i + 10]
        candidate_details = ' '.join(candidate_words)
        candidate_full_text = ' '.join([candidate_details, fixed_text]).strip()
        
        if len(tokenizer.encode(candidate_full_text, add_special_tokens=False)) <= token_limit:
            truncated_details = candidate_details
        else:
            break
    
    # If no 10-word chunk fits, try word by word
    if not truncated_details:
        for i in range(1, len(words)):
            candidate_details = ' '.join(words[:i])
            candidate_full_text = ' '.join([candidate_details, fixed_text]).strip()
            
            if len(tokenizer.encode(candidate_full_text, add_special_tokens=False)) <= token_limit:
                truncated_details = candidate_details
            else:
                break
    
    return truncated_details

def truncate_vehicle_text(vehicle_data: Dict[str, Any], token_limit: int, tokenizer) -> Tuple[Dict[str, Any], bool]:
    """Truncate vehicle text to stay within token limit."""
    # Check if truncation is needed
    current_tokens = get_total_token_count(vehicle_data, tokenizer)
    
    if current_tokens <= token_limit:
        return vehicle_data, False
    
    # Get the fixed parts (details_list and information_dict)
    fixed_text = extract_non_details_text(vehicle_data)
    
    # Check if fixed parts already exceed limit
    fixed_tokens = len(tokenizer.encode(fixed_text, add_special_tokens=False))
    if fixed_tokens > token_limit:
        print(f"Warning: Fixed text already exceeds limit ({fixed_tokens} > {token_limit})")
        return vehicle_data, False
    
    # Get details_text
    details_text = vehicle_data.get('details_text', '')
    
    # Try sentence-based truncation first
    truncated_details = truncate_by_sentences(details_text, fixed_text, token_limit, tokenizer)
    
    # If sentence-based truncation didn't work well, try word-based
    if not truncated_details or truncated_details == details_text:
        truncated_details = truncate_by_words(details_text, fixed_text, token_limit, tokenizer)
    
    # Update vehicle data
    updated_vehicle_data = vehicle_data.copy()
    updated_vehicle_data['details_text'] = truncated_details
    
    return updated_vehicle_data, True

print(f"Applying token limit of {TOKEN_LIMIT} tokens...")

Loading BERT tokenizer...
Applying token limit of 400 tokens...


In [6]:
# Apply token truncation
token_limited_data = {}
truncated_count = 0
over_limit_count = 0

for url, data in translated_data.items():
    processed_vehicle, was_truncated = truncate_vehicle_text(data, TOKEN_LIMIT, tokenizer)
    token_limited_data[url] = processed_vehicle
    
    if was_truncated:
        truncated_count += 1
        original_tokens = get_total_token_count(data, tokenizer)
        new_tokens = get_total_token_count(processed_vehicle, tokenizer)
        # logger.info(f"Truncated: {url} - Tokens: {original_tokens} -> {new_tokens}")

# Verify results
for url, data in token_limited_data.items():
    tokens = get_total_token_count(data, tokenizer)
    if tokens > TOKEN_LIMIT:
        over_limit_count += 1
        logger.warning(f"Still over limit: {url} ({tokens} tokens)")

print(f"\nStep 4 Complete:")
print(f"- Processed: {len(token_limited_data)} entries")
print(f"- Truncated: {truncated_count} entries")
print(f"- Percentage truncated: {truncated_count/len(token_limited_data)*100:.1f}%")
print(f"- Vehicles still over limit: {over_limit_count}")

Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 512). Running this sequence through the model will result in indexing errors



Step 4 Complete:
- Processed: 2510 entries
- Truncated: 357 entries
- Percentage truncated: 14.2%
- Vehicles still over limit: 0


## 6. Step 4: Field Cleaning and Formatting

Clean specific fields by:
1. Converting German-style numbers (dots to commas) in mileage
2. Removing Vehicle Identification No. fields
3. Removing KBA Key fields

Reason: The Vehicle Identification and KBA are individual vehicle numbers, and apart from the police, no one would search for exactly "the car that has the number XY" on the market. So we removed this info for our find-me-a-car model training.

In [7]:
# Apply field cleaning
final_data = {}
mileage_fixes = 0
removed_fields = 0

for vehicle_url, vehicle_data in token_limited_data.items():
    cleaned_entry = dict(vehicle_data)
    
    if 'information_dict' in cleaned_entry:
        info_dict = cleaned_entry['information_dict']

        # Convert German-style mileage numbers (dots to commas)
        if 'Read mileage' in info_dict:
            original_mileage = info_dict['Read mileage']
            # Replace dots with commas in numbers (e.g., "23.500" -> "23,500")
            info_dict['Read mileage'] = re.sub(r'(\d+)\.(\d+)', r'\1,\2', original_mileage)
            if info_dict['Read mileage'] != original_mileage:
                mileage_fixes += 1

        # Remove Vehicle Identification No. field
        if 'Vehicle Identification No.' in info_dict:
            del info_dict['Vehicle Identification No.']
            removed_fields += 1

        # Remove KBA Key fields (both manufacturer and type)
        keys_to_remove = []
        for key in info_dict.keys():
            if 'KBA' in key and 'Key' in key:
                keys_to_remove.append(key)

        for key in keys_to_remove:
            del info_dict[key]
            removed_fields += 1

    final_data[vehicle_url] = cleaned_entry

print(f"\nStep 4 Complete:")
print(f"- Processed: {len(final_data)} entries")
print(f"- Mileage format fixes: {mileage_fixes}")
print(f"- Removed fields: {removed_fields}")


Step 4 Complete:
- Processed: 2510 entries
- Mileage format fixes: 2475
- Removed fields: 6497


## 7. Save Final Results

Save the fully processed data to `vehicles_info.yaml` with proper formatting.

In [8]:
# Save the final processed data
print(f"Saving final processed data to: {output_file}")
with open(output_file, 'w', encoding='utf-8') as f:
    yaml.dump(final_data, f, default_flow_style=False, allow_unicode=True, sort_keys=True)

print(f"\n=== PROCESSING COMPLETE ===")
print(f"Input file: {input_file}")
print(f"Output file: {output_file}")
print(f"Total vehicles processed: {len(final_data)}")
print(f"Token limit: {TOKEN_LIMIT}")
print(f"\nProcessing Summary:")
print(f"- Step 1 (Deduplication): {modified_count} entries modified")
print(f"- Step 2 (Translation): {translation_count} entries translated")
print(f"- Step 3 (Field cleaning): {mileage_fixes} mileage fixes, {removed_fields} fields removed")
print(f"- Step 4 (Token limiting): {truncated_count} entries truncated")

# Show a sample of the final data structure
if final_data:
    sample_url = list(final_data.keys())[0]
    sample_data = final_data[sample_url]
    sample_tokens = get_total_token_count(sample_data, tokenizer)
    
    print(f"\nSample entry: {sample_url}")
    print(f"Token count: {sample_tokens}")
    print(f"Keys: {list(sample_data.keys())}")
    if 'details_text' in sample_data:
        details_preview = sample_data['details_text'][:100] + '...' if len(sample_data['details_text']) > 100 else sample_data['details_text']
        print(f"Details text preview: {details_preview}")

Saving final processed data to: /storage/courses/ds_workflow/car-web-scraping/data/vehicles_info.yaml

=== PROCESSING COMPLETE ===
Input file: /storage/courses/ds_workflow/car-web-scraping/data/vehicles_data.json
Output file: /storage/courses/ds_workflow/car-web-scraping/data/vehicles_info.yaml
Total vehicles processed: 2510
Token limit: 400

Processing Summary:
- Step 1 (Deduplication): 514 entries modified
- Step 2 (Translation): 1069 entries translated
- Step 3 (Field cleaning): 2475 mileage fixes, 6497 fields removed
- Step 4 (Token limiting): 357 entries truncated

Sample entry: https://autobid.de/en/item/mini-cooper-s-cabrio-3111666/details
Token count: 216
Keys: ['information_dict', 'details_list', 'details_text']
Details text preview: Demurrage chargesStarting on 22.07.2025 the daily demurrage for this item will be 6,00 EUR net, plus...
